In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
import scipy.optimize as sco
import matplotlib.pyplot as plt

date_dict = {
    '2019-1': pd.Timestamp('2019-03-31'),
    '2019-2': pd.Timestamp('2019-06-30'),
    '2019-3': pd.Timestamp('2019-09-30'),
    '2019-4': pd.Timestamp('2019-12-31'),
    '2020-1': pd.Timestamp('2020-03-31'),
    '2020-2': pd.Timestamp('2020-06-30'),
    '2020-3': pd.Timestamp('2020-09-30'),
    '2020-4': pd.Timestamp('2020-12-31'),
    '2021-1': pd.Timestamp('2021-03-31'),
    '2021-2': pd.Timestamp('2021-06-30'),
    '2021-3': pd.Timestamp('2021-09-30'),
    '2021-4': pd.Timestamp('2021-12-31'),
    '2022-1': pd.Timestamp('2022-03-31'),
    '2022-2': pd.Timestamp('2022-06-30'),
    '2022-3': pd.Timestamp('2022-09-30'),
    '2022-4': pd.Timestamp('2022-12-31'),
    '2023-1': pd.Timestamp('2023-03-31'),
    '2023-2': pd.Timestamp('2023-06-30'),
    '2023-3': pd.Timestamp('2023-09-30'),
    '2023-4': pd.Timestamp('2023-12-31'),
    '2024-1': pd.Timestamp('2024-03-31'),
    '2024-2': pd.Timestamp('2024-06-30'),
    '2024-3': pd.Timestamp('2024-09-30')
}
rf_dict = {
    '2019-1': 0.018,
    '2019-2': 0.018,
    '2019-3': 0.014,
    '2019-4': 0.0121,
    '2020-1': 0.0075,
    '2020-2': 0.0048,
    '2020-3': 0.0051,
    '2020-4': 0.0035,
    '2021-1': 0.0045,
    '2021-2': 0.0048,
    '2021-3': 0.005,
    '2021-4': 0.0051,
    '2022-1': 0.0053,
    '2022-2': 0.0109,
    '2022-3': 0.0124,
    '2022-4': 0.0137,
    '2023-1': 0.0175,
    '2023-2': 0.0204,
    '2023-3': 0.0246,
    '2023-4': 0.0237,
    '2024-1': 0.0224,
    '2024-2': 0.0236,
    '2024-3': 0.0226
}

df = pd.read_excel('KWI_adjustedQRanking.xlsx')
quarter = df.columns.tolist()
quarter.remove('Rankings')

def portfolio_performance(weights, returns, cov_matrix):
    portfolio_return = np.sum(returns.mean() * weights) * 12  # Annualized return
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(12)  # Annualized volatility
    return portfolio_return, portfolio_std_dev

def sharpe_ratio(weights, returns, cov_matrix, period):
    p_return, p_std_dev = portfolio_performance(weights, returns, cov_matrix)
    return (p_return - rf_dict[period]) / p_std_dev  # Sharpe ratio

def neg_sharpe(weights, returns, cov_matrix, period):
    return -sharpe_ratio(weights, returns, cov_matrix, period)

output_file = "Optimized_Portfolio_2.xlsx"

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    
    for period in quarter:
        symbols = [i + '.BK' for i in df[period]]
        
        # Check if "SCB.BK" is in the symbols list
        scb_data = None
        if "SCB.BK" in symbols:
            scb_data = pd.read_excel("Set50 Data/SCB_Price.xlsx", index_col=0, parse_dates=True)
            scb_data = scb_data.loc[:date_dict[period]]
        
        # Download data from Yahoo Finance
        data = yf.download(symbols, start='2004-01-01', end=date_dict[period], interval='1mo')['Close']
        
        # Merge SCB.BK data if applicable
        if scb_data is not None:
            data["SCB.BK"] = scb_data
        
        # Remove failed stocks
        data = data.loc[:, data.count() >= 60]
        valid_symbols = data.columns.tolist()
        
        if len(valid_symbols) == 0:
            print(f"No valid data for period {period}. Skipping...")
            continue
        
        # Calculate monthly returns
        returns = data.pct_change().dropna()
        cov_matrix = returns.cov()
        
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        bounds = tuple((0, 0.10) for _ in range(len(valid_symbols)))
        initial_weights = np.array([1. / len(valid_symbols)] * len(valid_symbols))
        
        optimal_weights = sco.minimize(neg_sharpe, initial_weights, args=(returns, cov_matrix, period),
                                       method='SLSQP', bounds=bounds, constraints=constraints)
        
        optimal_portfolio_return, optimal_portfolio_std_dev = portfolio_performance(optimal_weights.x, returns, cov_matrix)
        sharpe = sharpe_ratio(optimal_weights.x, returns, cov_matrix, period)
        
        portfolio_returns = (returns @ optimal_weights.x) * 100
        
        performance_df = pd.DataFrame(returns * 100)
        performance_df.insert(0, "Date", performance_df.index.date)
        performance_df["Portfolio"] = portfolio_returns
        
        weight_row = pd.DataFrame([["Weight"] + list(optimal_weights.x) + [1.00]],
                          columns=performance_df.columns)
        
        performance_df = pd.concat([weight_row, performance_df], ignore_index=True)
        performance_df.to_excel(writer, sheet_name=f"{period}_Performance", index=False)
        
        top_assets = sorted(zip(valid_symbols, optimal_weights.x), key=lambda x: x[1], reverse=True)[:5]
        top_holdings = ", ".join([f"{asset} ({weight:.2%})" for asset, weight in top_assets])
        
        summary_df = pd.DataFrame({
            "Metric": ["Annual Return", "Volatility", "Sharpe Ratio", "Number of Assets", "Top 5 Holdings"],
            "Value": [f"{optimal_portfolio_return:.2%}", f"{optimal_portfolio_std_dev:.2%}", f"{sharpe:.2f}", len(valid_symbols), top_holdings]
        })
        
        summary_df.to_excel(writer, sheet_name=f"{period}_Summary", index=False)
        
        workbook  = writer.book
        worksheet = workbook.get_worksheet_by_name(f"{period}_Performance")
        
        chart1 = workbook.add_chart({'type': 'line'})
        chart1.add_series({
            'name': 'Portfolio Returns',
            'categories': f"={period}_Performance!$A$2:$A${len(performance_df)}",
            'values': f"={period}_Performance!$G$2:$G${len(performance_df)}",
        })
        chart1.set_title({'name': 'Portfolio Performance'})
        chart1.set_x_axis({'name': 'Date'})
        chart1.set_y_axis({'name': 'Returns (%)'})
        
        worksheet.insert_chart('J2', chart1)

print(f"Portfolio optimization results saved to {output_file}")

[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed
[*********************100%***********************]  20 of 20 completed

1 Failed download:
['SCB.BK']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2004-01-01 -> 2021-03-31 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1072890000, endDate = 1617123600")')
[*********************100%***********************]  20 of 20 completed

1 Failed download:
['SCB.BK']: YFPricesMissingError('possibly deli

Portfolio optimization results saved to Optimized_Portfolio_2.xlsx


In [14]:
import yfinance as yf
import pandas as pd
symbols = [
    'ADVANC.BK', 'BBL.BK', 'BDMS.BK', 'BH.BK', 'EA.BK', 'GULF.BK', 
    'HMPRO.BK', 'KTB.BK', 'LH.BK', 'SAWAD.BK', 'TU.BK', 'CPF.BK', 
    'PTTEP.BK', 'MTC.BK', 'TISCO.BK', 'KTC.BK', 'KCE.BK', 'DELTA.BK'
]

# Download daily stock data
data = yf.download(['CBG.BK','WHA.BK'], start='2024-11-29', end='2024-11-30',group_by='tickers', interval='1h')
# scb_data = pd.read_excel("Set50 Data/SCB_Price.xlsx", index_col=0, parse_dates=True)
# scb_data = scb_data.loc[:date_dict[period]]
# if scb_data is not None:
#     data["SCB.BK"] = scb_data
data

[*********************100%***********************]  2 of 2 completed


Ticker                    WHA.BK                           CBG.BK         \
Price                       Open High   Low Close   Volume   Open   High   
Datetime                                                                   
2024-11-29 03:00:00+00:00   5.65  5.7  5.60  5.70        0  76.75  77.00   
2024-11-29 04:00:00+00:00   5.70  5.7  5.65  5.70  1164296  75.00  75.50   
2024-11-29 05:00:00+00:00   5.70  5.7  5.65  5.65  2204478  75.00  75.50   
2024-11-29 06:00:00+00:00   5.65  5.7  5.65  5.65   663230  75.50  75.50   
2024-11-29 07:00:00+00:00   5.65  5.7  5.65  5.70  2593679  75.25  76.75   
2024-11-29 08:00:00+00:00   5.65  5.7  5.65  5.65  4272698  76.00  76.50   
2024-11-29 09:00:00+00:00   5.70  5.7  5.65  5.65  5902182  76.50  76.75   

Ticker                                            
Price                        Low  Close   Volume  
Datetime                                          
2024-11-29 03:00:00+00:00  74.50  75.00        0  
2024-11-29 04:00:00+00:00  74.75  75.25  1299006  
2024-11-29 05:00:00+00:00  75.00  75.25   338254  
2024-11-29 06:00:00+00:00  75.50  75.50   367400  
2024-11-29 07:00:00+00:00  75.25  76.00  1875801  
2024-11-29 08:00:00+00:00  75.75  76.50   644001  
2024-11-29 09:00:00+00:00  76.25  76.75   710991